## Wstęp

Dzisiaj zapoznasz się z jednym z najważniejszych zagadnień w kryptografii - czyli z funkcjami skrótu. Opowiemy sobie czym one są, do czego służą i gdzie są wykorzystywane na przykładzie standardu SHA-3.

## Funkcje skrótu

Zwane też: funkcje skracające, funkcje mieszające, funkcje haszujące, a z języka ang. *hashing functions*. W słowniku języka polskiego nie ma oficjalnej nazwy tych funkcji, dlatego podałem wszystkie z którymi możesz się spotkać w literaturze.

Z natury są to funkcje jednokierunkowe, które jako wejście przyjmują ciąg binarny o potencjalnie nieograniczonej długości, a jako wyjście zwracają ciąg binarny zawsze tej samej długości:
$$
h: \{0, 1\}^* \rightarrow  \{0, 1\}^n
$$
gdzie *h* oznacza funkcję skrótu, a *n* to długość zwracanego skrótu. Ta matematyczna definicja nie uwzględnia jednak pewnych dodatkowych właściowści, które funkcja skótu powinna posiadać. Są to:
- skrót powinien być łatwy do obliczenia (wydajność),
- dysponując funkcją skrótu *h* oraz skrótem oznaczonym jako *h(x)* powinno być niewykonalne (a przynajmniej bardzo, bardzo trudne) znalezienie wartości *x*. Ta własność jest określana jako bezpieczeństwo pierwszego preobrazu (z ang. *first preimage resistance*),
- mając **podane** *h* oraz *x*, powinno być bardzo trudne odnalezienie wartości *y* takiej, że *h(x) = h(y)*. Ta własność jest określana jako bezpieczeństwo drugiego preobrazu (z ang. *second preimage resistance*),
- powinno być także trudno znaleźć dwie **dowolne** wartości *x* oraz *y* takie, że *h(x) = h(y)*. Ta własność zwana jest odpornością na kolizje (z ang. *collision resistance*).

Bezpieczeństwo funkcji skrótu podajemy w tzw. bitach bezpieczeństwa. Jeżeli funkcja skrótu posiada *b* bitów bezpieczeństwa odporności na dany atak, to znaczy, że atakujący musałby wykonać $2^b$ operacji aby atak się udał. Teoretyczne maksymalne wartości liczby bitów bezpieczeństwa to $n$ w przypadku ataków na pierwszy i drugi preobraz, oraz $\frac{n}{2}$ w przypadku szukaniu kolizji (ta wartość wynika z [paradoksu urodzin](https://en.wikipedia.org/wiki/Birthday_problem)), gdzie tak jak poprzednio *n* oznacza długość zwracanego skrótu. Mówiąc o bezpieczeństwie trzeba jednak pamiętać o jednym. Każda funkcja skrótu jest do złamania. Wynika to z prostego faktu: posiadamy teoretycznie nieograniczoną liczbę wejść, a ograniczoną (choć za zwyczaj jest to wartość duża) liczbę wyjść.

No dobrze, tylko gdzie my możemy funkcje skrótu wykorzystać? Obecnie wyszczególniłbym następujące schematy wykorzystujące funkcje skrótu:
- przechowywanie haseł w bazie danych,
- weryfikacja integralności danych (w szczególności schematy podpisu cyfrowego),
- kryptografia post-quantum, w szczególności gałąź zwana *hash-based cryptography*, o której więcej możesz przeczytać [tutaj](https://en.wikipedia.org/wiki/Hash-based_cryptography).

W historii były cztery światowe standardy: SHA-0, SHA-1, SHA-2 i SHA-3, gdzie skrót *SHA* oznacza z ang. *Secure Hash Algorithm*. Standaryzacją zajmuje się organizacja NIST (*National Institute of Standards and Technology*) ulokowana w USA. Mało popularny SHA-0 oraz znacznie popularniejszy SHA-1 są **niezalecane do użycia**. Zostały złamane. SHA-2 i SHA-3 to już nie pojedyncze funkcje skrótu, ale raczej ich rodziny. Oznaczenie SHA-256 mówi nam o funkcji skrótu z rodziny SHA-2 zwracającej 256 bitowy skrót. Analogicznie SHA3-256. Najbardziej popularne długości skrótu (choć oczywiście nie jedyne) to 256 i 512 bitów. Standardy SHA które nie zostały złamane i zaleca się je do użycia określamy mianem SHS (z ang. *Secure Hash Standard*).

## SHA-3

Po serii udanych ataków na funkcję SHA-1, które miały miejsce w 2004 i 2005 roku, świat został z w zasadzie jednym standardem, SHA-2. NIST i specjaliści z całego świata byli zgodni, że to ryzykowane nie mieć żadnej bezpiecznej alternatywy - stąd w 2007 roku ogłoszono konkurs na SHA-3. Konkurs miał 3 rundy i trwał dość długo, bo wpłynęły aż 64 zgłoszenia. Ostatecznie w 2012 roku ogłoszono, że zwycięzcą został algorytm Keccaka i to on stał się nowym standardem, SHA-3. Dzisiaj go zaprogramujemy.

Czy SHA-3 jest trudny do zaprogramowania? Powiem tak. Moim zdaniem, korzystając tylko z dokumentacji twórców trochę tak, bo jest kilka małych pułapek. Dlatego opiszę Ci krok po kroku co powinno się zrobić i będzie nieco łatwiej.

Dla zainteresowanych przydatne linki: [strona twórców](https://keccak.team/keccak_specs_summary.html), [NIST FIPS PUB 202](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.202.pdf) i ostatnia, **ważna** która przyda nam się na końcu ćwiczeń: [debug data](https://csrc.nist.gov/Projects/cryptographic-standards-and-guidelines/example-values) - interesuje nas wiersz SHA3-512 z tabeli.



### Krok 1 - stałe

SHA-3 jest bardzo wszechstronny, pozwala dopasować wersję algorytmu do długości słowa bitowego obsługiwanego przez nasz procesor. Dla najbardziej popularnej długości obecnie, czyli 64 bity, stosuje się wersję algorytmu zwaną Keccak-1600. Można stosować inne, opisane w NIST FIPS PUB 202, ale my kierujemy się wydajnością. 1600 oznacza, że w każdej rundzie algorytmu będziemy operować na 1600 bitach danych, a więc 200 bajtach. Ta liczba oczywiście jest nieprzypadkowa. Zauważ, że 200 bajtów, to dokładnie 25 słów bitowych, każde po 64 bity. Te 25 słów bitowych formowane jest w dwuwymiarową tablicę 5x5. Ta tablica zwana jest przez twórców *State*.

No dobrze, ale jak się do tego ma wiadomość? Tutaj z pomocą przychodzą dwie stałe: $r$ oraz $c$ opisane [tu](https://keccak.team/keccak.html). Stała $r$ oznacza *bit rate*, $c$ *capacity*, a ich wartości zależą od tego, jaką wersję algorytmu SHA-3 zastosujemy (my skupimy się na SHA3-512). SHA-3 działa na zasadzie funkcji gąbki, to znaczy w pętli głównej wchłaniana jest odpowiednia liczba bitów wiadomości do tablicy state, wykonujemy odpowiednie obliczenia, wchłaniamy kolejną porcję, liczymy itd. W naszym przypadku $r = 576$ bitów, a więc 72 bajty. $c$ to po prostu uzupełnienie, zauważ, że wchłaniana liczba bitów jest mniejsza niż rozmiar State, stąd dla $c = 1024$, $r + c = 1600$.

$n$ oznacza liczbę rund. Po wchłonięciu porcji danych $n$ razy wykonujemy zestaw funkcji opisanych przez twórców (o których później), gdzie $n = 12 + 2l$. Z tabeli 1 w dokumentacji NIST można wyczytać, że dla wersji Keccak-1600, $l = 6$, a więc nasza liczba rund to $n = 24$.

$RC$ to tablica 24 stałych, po jednej dla każdej z 24 rund, natomiast $rotations$ to tablica wartości, która mówi nam o ile mamy rotować słowo bitowe. Jest ona potrzebna w jednej funkcji z zestawu przedstawionego przez twórców.

Zadeklarujmy nasze stałe. Dla wygody wartości $r$ oraz $c$ zadeklarujemy jako liczbę bajtów. 

In [1]:
from math import floor

In [2]:
n = 24
r = 576 // 8
c = 1024 // 8

RC = [0x0000000000000001,
      0x0000000000008082,
      0x800000000000808A,
      0x8000000080008000,
      0x000000000000808B,
      0x0000000080000001,
      0x8000000080008081,
      0x8000000000008009,
      0x000000000000008A,
      0x0000000000000088,
      0x0000000080008009,
      0x000000008000000A,
      0x000000008000808B,
      0x800000000000008B,
      0x8000000000008089,
      0x8000000000008003,
      0x8000000000008002,
      0x8000000000000080,
      0x000000000000800A,
      0x800000008000000A,
      0x8000000080008081,
      0x8000000000008080,
      0x0000000080000001,
      0x8000000080008008]

rotations = [[0, 36, 3, 41, 18], [1, 44, 10, 45, 2], [62, 6, 43, 15, 61], [28, 55, 25, 21, 56], [27, 20, 39, 8, 14]]

### Krok 2 - Padding

Zauważ, że w każdej głównej iteracji algorytmu wchłaniamy 72 bajty wiadomości (wartość $r$). Jest bardzo mało prawdopodobne, że nasza wiadomość będzie akurat miała długość w bajtach równą 72 lub wielokrotności 72. Tutaj z pomocą przychodzi *padding*, czyli uzupełnianie wiadomości do wymaganej długości. Wskazówka: jako *wiadomość* rozum ciąg bajtów, czyli *bytearray* z którym będziemy robić sporo rzeczy.

SHA-3 stosuje metodę pad10*1, czyli w zamyśle dołącz do wiadomości bit 1, następnie odpowiednią liczbę bitów 0 i zamknij ciąg bitem 1. W SHA-3 ten algorytm jest wzbogacony o stałą 0x06:

1) Oblicz q = r - (len(msg) mod r). Zauważ, że $r$ i $len(m)$ zwracają długości w bajtach.

2) Jeżeli q = 1 wykonaj: dołącz do wiadomości bajt 0x81.

3) Jeżeli q = 2 wykonaj: dołącz do wiadomości bajt 0x06, a następnie dołącz do wiadomości bajt 0x80.

4) Jeżeli q > 2 wykonaj: dołącz do wiadomości bajt 0x06, następnie dołącz do wiadomości dokładnie $q - 2$ bajty 0x00 i na końcu dołącz do wiadomości bajt 0x80.

Sprawdź reprezentację bitową 0x80 oraz 0x81, jak wspominałem stała 0x06 to już dodatek autorów. Czy widzisz w powyższym algorytmie coś dziwnego? Jakiego porządku bitów używa SHA-3? W każdym przypadku, po operacji paddingu len(msg) powinno być wielokrotnością 72.

**Zadanie 1**

Napisz ciało metody pad10*1. Parametrem metody jest tablica bajtów wiadomości (bytearray).

In [3]:
def pad(msg):
    msg.reverse()
    q = r - (len(msg) % r)
    if(q==1):
        msg.append(int(0x86))
    else:
        msg.append(int(0x06))
        for i in range(q-2):
            msg.append(int(0x00))
        msg.append(int(0x80))
    return msg

In [4]:
# EXAMPLE PADDING

msg = 'abb'
msg = [ord(i) for i in msg]
print(msg)
msg = pad(msg)
print(msg)

[97, 98, 98]
[98, 98, 97, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128]


### Krok 3 - funkcje pomocnicze

Będziemy potrzebowali operować na wartości State na dwa sposoby: raz jako na tablicy bajtów o długości 1600 (potrzebne przy wchłanianiu danych), a raz jako na dwuwymiarowej tablicy 64-bitowych słów o rozmiarach 5x5 (potrzebne w dalszej części). Musimy napisać łącznie trzy funkcje pomocnicze:

1) convertToInts(state) - ta funkcja powinna pobierać State w formie wektora bajtów i zwracać tablicę 5x5. Algorytm jest taki, że licząc od początku pobieramy po 8 bajtów (64 bity) i przypisujemy do nowego State'a (oznaczmy go jako State'), ale uwaga, dane układamy wierszami. A więc bajty 0-7 wektora State konwertujemy na liczbę i przypisujemy do State'[0][0], bajty 8-15 ze State przypisujemy jako liczbę do State'[1][0] itd. Dodatkowo, konwertując bajty na liczby używamy porządku **Little Endian**.

2) convertToBytes(state) - ta funkcja pobiera State jako dwuwymiarową tablicę i konwertuje z poworotem na wektor bajtów State' (nowy State). Dane również pobieramy wierszami i dekodujemy w porządku **Little Endian**. A więc liczba przechowywana w State[0][0] jest kodowana jako 8 pierwszych bajtów State', liczba w State[1][0] jako 8 kolejnych bajtów itd. Czyli robimy funkcję odwrotną do poprzedniej. Funkcja zwraca State' o rozmiarze 1600 bajtów.

3) rot(a, n) - ta funkcja dokonuje bitowej rotacji 64 bitowej liczby $a$ w lewo o $n$ bitów.

**Zadanie 2**

Uzupełnij ciała metod convertToBytes oraz convertToInts zgodnie z powyższym opisem.

In [5]:
def to_stream(A):
    num = 0
    for i in range(8):
        num = num + (256**i) * A[i]
    return num

def from_stream(num):
    A = [0 for i in range(8)]
    for i in range(8):
        A[i] = num%256
        num = (num-A[i])//256
        if(num==0):
            break
    return A

def convertToInts(state):
    table = [[0 for i in range(5)] for i in range(5)]
    for idx in range(0,200,8):
        bytes_8 = state[idx:idx+8]
        bytes_8 = to_stream(bytes_8)
        table[(idx%40)//8][floor(idx/40.0)] = bytes_8
    return table

def convertToBytes(table):
    l = 5
    state = []
    for i in range(l):
        for j in range(l):
            bytes_8 = from_stream(table[j][i])
            for byte in bytes_8:
                state.append(byte)
    return state

def rot(x, n):
    return int(f"{x:064b}"[n:] + f"{x:064b}"[:n], 2)

def rot_list(A, n):
    a = to_stream(A)
    r = rot(a,n)
    return from_stream(r)


### Krok 4 - permutacje

SHA-3 wykorzystuje zestaw pięciu permutacji, które wykorzystywane są w każdej pojedynczej rundzie (*rundzie*, nie iteracji pętli głównej algorytmu). Nie są one skomplikowane, dlatego nie będziemy pisali oddzielnej funkcji do każdej z nich. Każda z nich korzysta ze State'a jako tablicy liczby 5x5.

1) Krok $Θ$ wykonaj:
- C[i] = state[i,0] xor state[i,1] xor state[i,2] xor state[i,3] xor state[i,4] dla i = 0,...,4. C to wektor pomocniczy.
- D[i] = C[i-1] xor rot(C[i+1], 1) dla i = 0,...,4, D to wektor pomocniczy.
- state[i][j] = state[i][j] xor D[i], dla i,j = 0,...,4.

2) Kroki $ρ$ oraz $π$ wykonaj: B[j][2 * i + 3 * j] = rot(state[i][j], rotations[i][j]), B to zmienna pomocnicza.

3) Krok $Χ$ wykonaj: state[i][j] = B[i][j] ^ ((~B[i + 1][j]) & B[i + 2][j]).

4) Krok $ι$ wykonaj: state[0][0] = RC. RC to stała, którą będziemy przekazywać do funkcji realizującej te permutacje.

Ostatecznie mamy do napisania dwie funkcje w pythonie: *compute* i *oneRound*.

Funkcja *compute(state)* przyjmuje jako parametr State w formie wektora bajtów i wykonuje następujące czynności:

1) Przekonwertuj State do formy tablicy liczb.

2) Wykonuj *state = oneRound(state, RC[i])* dla i = 0,...,n-1

3) Przekonwertuj State do wektora bajtów.

4) Zwróc State.

Funkcja *oneRound(state, RC)* wykonuje obliczenia dla pojedynczej rundy:

1) Utwórz niezbędne zmienne pomocnicze.

2) Wykonaj krok $Θ$.

3) Wykonaj kroki $ρ$ oraz $π$.

4) Wykonaj krok $Χ$.

5) Wykonaj krok $ι$.

6) Zwróć State.

**Zadanie 3**

Uzupełnij ciała metod *compute* i *oneRound* zgodnie z algorytmem opisanym powyżej.

In [44]:
def ones(n):
    return bin((2**n)-1)

def NOT(x, l=64):
    y = int(ones(l),2)
    return x ^ y


In [47]:
def logger(msg):
    print(msg)
    # pass

def compute(state):
    state = convertToInts(state)
    for i in range(n-1):
        state = oneRound(state, RC[i])
    state = convertToBytes(state)
    return state

def theta(state, l=5):
    C = [(state[i][0] ^ state[i][1] ^ state[i][2] ^ state[i][3] ^ state[i][4]) for i in range(l)]
    D = [(C[(i-1)%l] ^ rot(C[(i+1)%l], 1)) for i in range(l)]
    B = [[0 for j in range(l)] for i in range(l)]

    for i in range(l):
        for j in range(l):
            B[i][j] = (state[i][j] ^ D[i])
    return B

def rho(state, l=5):
    B = [[0 for j in range(l)] for i in range(l)]
    B[0][0] = state[0][0]
    x,y = 1,0
    for t in range(24):
        B[x][y] = rot(state[x][y], ((t+1)*(t+2)//2)%64)
        x, y = y, (2*x+3*y)%l
    return B

def pi(state, l=5):
    B = [[0 for j in range(l)] for i in range(l)]
    for i in range(l):
        for j in range(l):
            B[i][j] = state[(i+3*j)%l][i]
    return B

def chi(state, l=5):
    B = [[0 for j in range(l)] for i in range(l)]
    for i in range(l):
        for j in range(l):
            B[i][j] = state[i][j] ^ (NOT(state[(i+1)%l][j]) & state[(i+2)%l][j])
    return B

def oneRound(state, rc):
    l = 5
    # Stage Θ
    state = theta(state)
    logger(f"state after theta:")
    show_state(state)

    # Stage ρ  
    state = rho(state)
    logger(f"state after rho:")
    show_state(state)

    # Stage π 
    state = pi(state)
    logger(f"state after pi:")
    show_state(state)

    # Stage Χ
    state = chi(state)
    logger(f"state after chi:")
    show_state(state)
            
    #  Stage ι
    state[0][0] = rc
    logger(f"state after iota:")
    show_state(state)
    
    return state
    

def show_state(state):
    s = convertToBytes(state)
    i=0
    h=16
    while(i<200):
        if(i+h>200):
            print(s[i:])
            break
        else:
            print(s[i:(i+h)])
            i+=h
    print("---")

def show_stream(stream):
    i=0
    while(i<200):
        s = bytes_state[i:min(i+16,200)]
        for b in s:
            print(hex(b), end = ' ')
        print("\n")
        i = i+16
    print("---")
    

### Krok 5 - główna pętla algorytmu

Wreszcie możemy napisać funkcję główną liczącą skrót *sha3-512*. Przyjmuje ona jako parametr bajty wiadomości $m$ i zwraca bajty skrótu:

1) Dokonaj paddingu wiadomości.

2) Wchłoń $r$ bajtów wiadomości do State'a. "Wchłanianie" oznacza tu operację XOR kolejnych bajtów wiadomości z $r$ początkowymi bajtami State'a.

3) Wywołaj metodę *compute(state)*.

4) Powtarzaj kroki 2 i 3 dopóki cała wiadomość nie zostanie wchłonięta.

5) Zwróć pierwsze 64 bajty State'a. To jest nasz 512-bitowy skrót.

**Zadanie 4**

Uzupełnij kod poniższej metody tak, aby zwracała skrót z wiadomości. Przetestuj tę metodę, czy działa prawidłowo. Tablicę bajtów przez nią zwróconą trzeba przekonwertować na ciąg znaków hexadecymalnych i wypisać na ekranie. Przetestuj funkcję dla poniższych ciągów znakowych:

1) ""

2) "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

3) "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

Porównaj, czy skróty zgadzają się ze zwróconymi z [tego](https://www.browserling.com/tools/sha3-hash) kalkulatora lub zaimportuj bibliotekę hashlib i porównaj swoje wyniki z jej wynikami.

In [39]:
def to_64_bytes(A, n=64, byte_num=8):
    s = 2**n
    num = 0
    i,j = 0,0
    for k in range(byte_num):
        num = num + (s**k) * A[i][j]
        j = (j+1)%5
        if(j==0):
            i = i+1
    return num    

def sha3_512(msg):
    msg = [ord(i) for i in msg]
    msg = pad(msg)
    state = [0x0 for i in range(200)]
    logger(f">initial state:\n{state}")
    
    idx = 0
    while(idx<len(msg)):
        for j in range(0,r):
            state[j] = state[j] ^ msg[idx+j]
        logger(f">XOR'ed state:\n{state}")
        logger(">compute state:")
        state = compute(state)
        logger(f">computed state:\n")
        show_stream(state)
        idx = idx+r
        break
    return hex(to_64_bytes(convertToInts(state), 64, 8))

In [48]:
# INITIAL TEST
msg = ''
print(msg)
out = sha3_512(msg)
print(f"OUT = \n{out}")


>initial state:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
>XOR'ed state:
[6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

**Zadanie 5**

Porównaj szybkość działania Twojej implementacji SHA3-512 z implementacjami SHA3-512 oraz SHA-512 dostępnymi w bibliotece hashlib. Postaraj się aby porównanie uwzględniało zarówno dane bardzo małe, jak i duże (na ile Ci pozwoli collab), a wyniki przedstaw na wykresie.

In [111]:
import hashlib
import random
import string

def sha_hashlib(message):
    hashlib_sha = hashlib.sha256()
    hashlib_sha.update(message.encode('ascii'))
    result = hashlib_sha.digest()
    result = int.from_bytes(result, "little")
    return result

def compare(msg):
    # print(f"For message:\n{msg}\nResults:\n{sha3_512(msg)}\n{sha_hashlib(msg)}")
    return sha3_512(msg), sha_hashlib(msg)


def test_random(c=100, l=100):
    correct, incorrect = 0,0
    for i in range(c):
        msg = ''.join(random.choices(string.ascii_lowercase, k=l))
        # print(msg)
        a, b = compare(msg)
        if(a==b):
            correct +=1
        else:
            incorrect +=1
    return f"CORRECT: {correct}, INCORRECT: {incorrect}"

print(test_random(10))



CORRECT: 0, INCORRECT: 10


In [114]:
# Compare for "", along with the online generator

# a69f73cca23a9ac5c8b567dc185a756e97c982164fe25859e0d1dcc1475c80a615b2123af1f5f94c11e3e9402c3ac558f500199d95b6d3e301758586281dcd26
# 8726747316830419364958639139147315824775555516067481895052668517585674312541027591056254248436504998721202987065792642411491591677611800488385866310208806
compare("")

(5435703591570960776260562982280818020785286267959326542574802805698553056395806552613891950360599158320678341781548244062209552509758322045863306454892545,
 38772261170797515502142737251560910253885555854579348417967781179871348437219)

**Zadanie 6**

Przerób powyższy kod tak, aby stał się funkcją SHA3-256.

In [112]:
# zwraca 256 bitów, a nie 512

def sha_256(msg):
    msg = [ord(i) for i in msg]
    
    msg = pad(msg)
    state = [0x0 for i in range(200)]
    
    idx = 0
    while(idx<len(msg)):
        for j in range(0,r):
            state[j] = state[j] ^ msg[idx+j]
        state = compute(state)
        idx = idx+r
        break
    return to_64_bytes(convertToInts(state), 64, 4)

In [113]:
# INITIAL TEST
msg = ''
out = sha_256(msg)
print(f"OUT = \n{out}")

OUT = 
16667725809947548670772731039916771448816485288592010933098935198940529688577
